## Part I. Scraping Table From Wiki Page

In [1]:
import pandas as pd
import numpy as np


#reading html page for table
tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M", header = 0)


table = tables[0]                      #identifying the table 
df = pd.DataFrame(table)               #converting table to dataframe
df = df[df.Borough != 'Not assigned']  #dropping records where borough is not assigned
df.sort_values('Postcode')             #sorting to see multiple neighborhoods with the same postcode


#identifying neighborhoods with the same postcode and convert to a list
lst = []
nbhds = []
for pc in df['Postcode']:
    lst = df.loc[df['Postcode'] == pc]['Neighbourhood'].tolist()
    nbhds.append(', '.join(lst))


df['Neighbourhood'] = nbhds            #listing neighborhoods with the same postcode together
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough'] #assign "Not assigned" neighborhoods the same name as their borough
df.drop_duplicates(inplace = True)     #drop dupicate rows
df = df.reset_index(drop = True)       #reset index
df.head(10)                            #display first 10 rows table

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


## Part II. Geographical Coordinates For Each Neighborhood

In [3]:
df_geospatial = pd.read_csv('https://cocl.us/Geospatial_data')               #importing csv
df_geospatial.rename(columns = {'Postal Code': 'Postcode'}, inplace = True)  #rename post code column to match with table above
df = df.join(df_geospatial.set_index('Postcode'), on='Postcode')             #join the dataframe on "postcode" to include the geospatial coords
df.head(10)                                                                  #show first 10 rows of new dataframe

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


## Part III. Explore/Cluster Toronto Data

In [33]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - anaconda/linux-64::conda-build==3.17.8=py36_0
  - anaconda/linux-64::grpcio==1.16.1=py36hf8bcb03_1
  - anaconda/linux-64::keras==2.1.5=py36_0
  - anaconda/linux-64::libarchive==3.3.3=h5d8350f_5
  - anaconda/linux-64::python-libarchive-c==2.8=py36_6
  - anaconda/linux-64::tensorboard==1.8.0=py36hf484d3e_0
  - anaconda/linux-64::tensorflow==1.8.0=h57681fa_0
  - anaconda/linux-64::tensorflow-base==1.8.0=py36h5f64886_0
  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::bottleneck==1.2.1=py37h035aef0_1
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::datashape==0.5.4=py37_1
  - defaults/linux-64::mkl-service==1.1.2=py37h9

In [35]:
#import dependencies
import folium 
from geopy.geocoders import Nominatim
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import json
from pandas.io.json import json_normalize

In [36]:
#idenitfy foursquare API credentials
CLIENT_ID = 'ELSG32TONI3DCWBV45RZO5AH5TXQRPEANXTYBZHYD4PBN12G' # your Foursquare ID
CLIENT_SECRET = 'H2XB5ENY1CSCUYOQHNWJIPLJPJ5L2V34A4QP0EA0J4LCVP0I' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ELSG32TONI3DCWBV45RZO5AH5TXQRPEANXTYBZHYD4PBN12G
CLIENT_SECRET:H2XB5ENY1CSCUYOQHNWJIPLJPJ5L2V34A4QP0EA0J4LCVP0I


#### Getting the geographical coordinates of Toronto

In [38]:
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Creating map of Toronto with Neighborhood names

In [82]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Getting Top 100 venues within 5000 meters of Queen's Park 

In [81]:
nbhd = 'Queen\'s Park' 
mask = df['Neighborhood'].str.contains(nbhd)
nbhd_lat = df.loc[mask, 'Latitude'].iat[0]
nbhd_long = df.loc[mask, 'Longitude'].iat[0]

LIMIT = 100

radius = 5000 
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    nbhd_lat, 
    nbhd_long, 
    radius, 
    LIMIT)

results = requests.get(url).json()
print('The results are for the neighbordhood -', nbhd, 'located at', nbhd_lat, nbhd_long)

The results are for the neighbordhood - Queen's Park located at 43.6623015 -79.3894938


In [83]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Getting venues nearby Queen's Park

In [84]:
#create dataframe from results
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,The Alley,Bubble Tea Shop,43.665922,-79.385567
1,DanceLifeX Centre,Dance Studio,43.666956,-79.385297
2,Banh Mi Boys,Sandwich Place,43.659188,-79.382131
3,Hailed Coffee,Coffee Shop,43.658833,-79.383684
4,Jimmy's Coffee,Coffee Shop,43.658421,-79.385613


In [85]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


#### Getting all venues in Toronto

In [86]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [87]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'], latitudes=df['Latitude'],longitudes=df['Longitude'])
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

Parkwoods
Victoria Village
Harbourfront, Regent Park
Lawrence Heights, Lawrence Manor
Queen's Park
Islington Avenue
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The D

In [88]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 


toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()


num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    #print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    #print('\n')

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

----Adelaide, King, Richmond----
----Agincourt----
----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
----Alderwood, Long Branch----
----Bathurst Manor, Downsview North, Wilson Heights----
----Bayview Village----
----Bedford Park, Lawrence Manor East----
----Berczy Park----
----Birch Cliff, Cliffside West----
----Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe----
----Brockton, Exhibition Place, Parkdale Village----
----Business Reply Mail Processing Centre 969 Eastern----
----CFB Toronto, Downsview East----
----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
----Cabbagetown, St. James Town----
----Caledonia-Fairbanks----
----Canada Post Gateway Processing Centre----
----Cedarbrae----
----Central Bay Street----
----Chinatown, Grange Park, Kensington Market----
----Christie----
---

#### Getting Top 100 venues in each neighborhood

In [89]:
num_top_venues = 100

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,91th Most Common Venue,92th Most Common Venue,93th Most Common Venue,94th Most Common Venue,95th Most Common Venue,96th Most Common Venue,97th Most Common Venue,98th Most Common Venue,99th Most Common Venue,100th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,Bar,Burger Joint,Asian Restaurant,Restaurant,American Restaurant,...,Empanada Restaurant,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Costume Shop,Climbing Gym,Convenience Store,BBQ Joint,Bike Rental / Bike Share
1,Agincourt,Skating Rink,Breakfast Spot,Lounge,Clothing Store,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,...,Arts & Crafts Store,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service,Airport Lounge,Airport Gate
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,...,Arts & Crafts Store,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service,Airport Lounge,Airport Gate
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Fast Food Restaurant,Japanese Restaurant,Pharmacy,Coffee Shop,Sandwich Place,Discount Store,Beer Store,Fried Chicken Joint,...,Asian Restaurant,Arts & Crafts Store,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service,Airport Lounge
4,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Skating Rink,Sandwich Place,Pub,Pool,Pharmacy,Gym,Comic Shop,...,Athletics & Sports,Hakka Restaurant,Asian Restaurant,Arts & Crafts Store,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal


#### Using k-Means to cluster venues into 5 clusters

In [90]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df
toronto_merged.rename(columns = {'Neighbourhood':'Neighborhood'}, inplace = True)


# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,91th Most Common Venue,92th Most Common Venue,93th Most Common Venue,94th Most Common Venue,95th Most Common Venue,96th Most Common Venue,97th Most Common Venue,98th Most Common Venue,99th Most Common Venue,100th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,4.0,Food & Drink Shop,Park,Fast Food Restaurant,Event Space,...,Arts & Crafts Store,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service,Airport Lounge,Airport Gate
1,M4A,North York,Victoria Village,43.725882,-79.315572,3.0,Pizza Place,French Restaurant,Portuguese Restaurant,Hockey Arena,...,Asian Restaurant,Arts & Crafts Store,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service,Airport Lounge
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,3.0,Coffee Shop,Bakery,Park,Café,...,Baseball Stadium,Baseball Field,Bar,Bagel Shop,Baby Store,BBQ Joint,Auto Garage,Boat or Ferry,Athletics & Sports,Asian Restaurant
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,3.0,Accessories Store,Coffee Shop,Boutique,Miscellaneous Shop,...,Asian Restaurant,Arts & Crafts Store,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service,Airport Lounge
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,3.0,Coffee Shop,Park,Gym,Diner,...,Baseball Stadium,Baseball Field,Bank,Bakery,Bagel Shop,Baby Store,BBQ Joint,Auto Workshop,Athletics & Sports,Bike Rental / Bike Share


#### Creating map

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

#### Examine Clusters

In [91]:
cluster1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]
cluster2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]
cluster3 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]
cluster4 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]
cluster5 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

In [92]:
cluster1

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,91th Most Common Venue,92th Most Common Venue,93th Most Common Venue,94th Most Common Venue,95th Most Common Venue,96th Most Common Venue,97th Most Common Venue,98th Most Common Venue,99th Most Common Venue,100th Most Common Venue
35,East Toronto,Pizza Place,Convenience Store,Park,Yoga Studio,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant,...,Arts & Crafts Store,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service,Airport Lounge,Airport Gate
63,"The Junction North, Runnymede",Pizza Place,Convenience Store,Grocery Store,Bus Line,Yoga Studio,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,...,Arts & Crafts Store,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service,Airport Lounge,Airport Gate
64,Weston,Convenience Store,Yoga Studio,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,...,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service,Airport Lounge,Airport Gate,Airport Food Court
66,York Mills West,Convenience Store,Bank,Park,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,...,Arts & Crafts Store,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service,Airport Lounge,Airport Gate


In [93]:
cluster2

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,91th Most Common Venue,92th Most Common Venue,93th Most Common Venue,94th Most Common Venue,95th Most Common Venue,96th Most Common Venue,97th Most Common Venue,98th Most Common Venue,99th Most Common Venue,100th Most Common Venue
11,"Cloverdale, Islington, Martin Grove, Princess ...",Bank,Yoga Studio,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,...,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service,Airport Lounge,Airport Gate,Airport Food Court
12,"Highland Creek, Rouge Hill, Port Union",Bar,Yoga Studio,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,...,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service,Airport Lounge,Airport Gate,Airport Food Court
34,"Northwood Park, York University",Massage Studio,Falafel Restaurant,Coffee Shop,Bar,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,...,Arts & Crafts Store,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service,Airport Lounge,Airport Gate


In [94]:
cluster3 #bars

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,91th Most Common Venue,92th Most Common Venue,93th Most Common Venue,94th Most Common Venue,95th Most Common Venue,96th Most Common Venue,97th Most Common Venue,98th Most Common Venue,99th Most Common Venue,100th Most Common Venue
1,Victoria Village,Pizza Place,French Restaurant,Portuguese Restaurant,Hockey Arena,Coffee Shop,Drugstore,Discount Store,Dive Bar,Dog Run,...,Asian Restaurant,Arts & Crafts Store,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service,Airport Lounge
2,"Harbourfront, Regent Park",Coffee Shop,Bakery,Park,Café,Gym / Fitness Center,Pub,Mexican Restaurant,Restaurant,Breakfast Spot,...,Baseball Stadium,Baseball Field,Bar,Bagel Shop,Baby Store,BBQ Joint,Auto Garage,Boat or Ferry,Athletics & Sports,Asian Restaurant
3,"Lawrence Heights, Lawrence Manor",Accessories Store,Coffee Shop,Boutique,Miscellaneous Shop,Furniture / Home Store,Clothing Store,Event Space,Vietnamese Restaurant,Costume Shop,...,Asian Restaurant,Arts & Crafts Store,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service,Airport Lounge
4,Queen's Park,Coffee Shop,Park,Gym,Diner,Yoga Studio,Restaurant,Italian Restaurant,Japanese Restaurant,Fast Food Restaurant,...,Baseball Stadium,Baseball Field,Bank,Bakery,Bagel Shop,Baby Store,BBQ Joint,Auto Workshop,Athletics & Sports,Bike Rental / Bike Share
6,"Rouge, Malvern",Fast Food Restaurant,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,...,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service,Airport Lounge,Airport Gate,Airport Food Court
7,Don Mills North,Café,Caribbean Restaurant,Gym / Fitness Center,Baseball Field,Japanese Restaurant,Yoga Studio,Donut Shop,Dive Bar,Dog Run,...,Arts & Crafts Store,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service,Airport Lounge,Airport Gate
8,"Woodbine Gardens, Parkview Hill",Pizza Place,Fast Food Restaurant,Gastropub,Bank,Intersection,Athletics & Sports,Breakfast Spot,Pharmacy,Gym / Fitness Center,...,Asian Restaurant,Arts & Crafts Store,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service,Airport Lounge
9,"Ryerson, Garden District",Coffee Shop,Clothing Store,Cosmetics Shop,Café,Fast Food Restaurant,Ice Cream Shop,Lingerie Store,Bookstore,Restaurant,...,Field,Farmers Market,Falafel Restaurant,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Doner Restaurant
10,Glencairn,Pizza Place,Park,Japanese Restaurant,Sushi Restaurant,Pub,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,...,Athletics & Sports,Asian Restaurant,Arts & Crafts Store,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service
13,"Flemingdon Park, Don Mills South",Gym,Beer Store,Asian Restaurant,Coffee Shop,Italian Restaurant,Discount Store,Dim Sum Restaurant,Restaurant,Japanese Restaurant,...,Auto Workshop,Auto Garage,Athletics & Sports,Arts & Crafts Store,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal


In [95]:
cluster4

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,91th Most Common Venue,92th Most Common Venue,93th Most Common Venue,94th Most Common Venue,95th Most Common Venue,96th Most Common Venue,97th Most Common Venue,98th Most Common Venue,99th Most Common Venue,100th Most Common Venue
0,Parkwoods,Food & Drink Shop,Park,Fast Food Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Falafel Restaurant,Eastern European Restaurant,...,Arts & Crafts Store,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service,Airport Lounge,Airport Gate
21,Caledonia-Fairbanks,Park,Fast Food Restaurant,Women's Store,Market,Pharmacy,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,...,Arts & Crafts Store,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service,Airport Lounge,Airport Gate
32,Scarborough Village,Playground,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,...,Arts & Crafts Store,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service,Airport Lounge,Airport Gate
40,"CFB Toronto, Downsview East",Airport,Park,Yoga Studio,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,...,Asian Restaurant,Arts & Crafts Store,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service,Airport Lounge
61,Lawrence Park,Bus Line,Park,Swim School,Yoga Studio,Donut Shop,Discount Store,Dive Bar,Dog Run,Doner Restaurant,...,Arts & Crafts Store,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service,Airport Lounge,Airport Gate
68,"Forest Hill North, Forest Hill West",Jewelry Store,Park,Sushi Restaurant,Trail,Yoga Studio,Donut Shop,Discount Store,Dive Bar,Dog Run,...,Asian Restaurant,Arts & Crafts Store,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service,Airport Lounge
85,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,...,Arts & Crafts Store,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service,Airport Lounge,Airport Gate
91,Rosedale,Park,Playground,Building,Trail,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,...,Asian Restaurant,Arts & Crafts Store,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service,Airport Lounge
98,"The Kingsway, Montgomery Road, Old Mill North",River,Park,Yoga Studio,Donut Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,...,Arts & Crafts Store,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service,Airport Lounge,Airport Gate


In [97]:
cluster5

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,91th Most Common Venue,92th Most Common Venue,93th Most Common Venue,94th Most Common Venue,95th Most Common Venue,96th Most Common Venue,97th Most Common Venue,98th Most Common Venue,99th Most Common Venue,100th Most Common Venue
